# FIRST OUTLINE THINGS TO DO
## Connect To API
## Pull all features
## Add feature to column in DF
## Write DF to an Excel


In [1]:
import requests
import pandas as pd

# Define the API endpoint and query parameters
url = "https://search.rcsb.org/rcsbsearch/v2/query"
query = {
    "query": {
        "type": "terminal",
        "service": "text",
        "parameters": {
            "attribute": "rcsb_accession_info.deposit_date",
            "operator": "exists"
        }
    },
    "return_type": "entry",
    "request_options": {
        "paginate": {
            "start": 0,
            "rows": 10000
        }
    }
}

# Send the request to the RCSB PDB API
response = requests.post(url, json=query)
data = response.json()

# Check the structure of the response
#print(data)

# Extract the list of PDB IDs if 'result_set' is present
if 'result_set' in data:
    pdb_ids = [entry['identifier'] for entry in data['result_set']]
    accession_numbers = [entry['rcsb_accession_info']['accession_number'] for entry in data['result_set'] if 'rcsb_accession_info' in entry]
    # Convert the list to a DataFrame
    df = pd.DataFrame(pdb_ids, columns=['PDB_ID'])
    # Filter out CSM PDBs (assuming CSM PDBs have a specific pattern in their IDs)
    non_csm_df = df[~df['PDB_ID'].str.contains('CSM')]
    # Display the DataFrame
    display(non_csm_df)
else:
    print("The key 'result_set' was not found in the response.")


,PDB_ID
0,100D
1,101D
2,101M
3,102D
4,102L
...,...
9995,1ID7
9996,1ID8
9997,1ID9
9998,1IDA


In [5]:
import requests
import pandas as pd

def pdb_to_uniprot(pdb_id):
    # Define the API endpoint for the specific PDB ID
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id}"
    
    # Send the request to the PDBe API
    response = requests.get(url)
    if response.status_code == 404:
        print(f"PDB ID {pdb_id} not found (HTTP Error 404).")
        return None
    
    content = response.json()
    
    # Extract UniProt ID
    uniprot_ids = []
    if pdb_id.lower() in content:
        for uniprot in content[pdb_id.lower()]['UniProt'].keys():
            uniprot_ids.append(uniprot)
    
    return uniprot_ids

# Example usage
pdb_id = "2o14"  # Replace with your PDB ID
uniprot_ids = pdb_to_uniprot(pdb_id)
print(f"UniProt IDs for PDB ID {pdb_id}: {uniprot_ids}")


UniProt IDs for PDB ID 2o14: ['P42304']


In [11]:
import requests

def get_protein_info(uniprot_id):
    # Define the API endpoint for the specific UniProt ID
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    
    # Send the request to the UniProt API
    response = requests.get(url)
    data = response.json()
    
    return data

def get_protein_info(uniprot_id):
    # Define the API endpoint for the specific UniProt ID
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    
    # Send the request to the UniProt API
    response = requests.get(url)
    data = response.json()
    
    return data

def extract_ec_numbers(protein_info):
    ec_numbers = []
    for comment in protein_info.get('comments', []):
        if comment.get('commentType') == 'CATALYTIC ACTIVITY':
            reaction = comment.get('reaction', {})
            ec_number = reaction.get('ecNumber')
            if ec_number:
                ec_numbers.append(ec_number)
    return ec_numbers

# Example usage
uniprot_id = "P00330"
protein_info = get_protein_info(uniprot_id)
ec_numbers = extract_ec_numbers(protein_info)

# Display some key information about the protein
print(f"Protein Name: {protein_info['proteinDescription']['recommendedName']['fullName']['value']}")
print(f"Organism: {protein_info['organism']['scientificName']}")
print(f"Gene Name: {protein_info['genes'][0]['geneName']['value']}")
print(f"The EC numbers for UniProt ID {uniprot_id} are: {ec_numbers}")



Protein Name: Alcohol dehydrogenase 1
Organism: Saccharomyces cerevisiae (strain ATCC 204508 / S288c)
Gene Name: ADH1
The EC numbers for UniProt ID P00330 are: ['1.1.1.1', '1.1.1.1', '1.1.1.1', '1.1.1.54', '1.1.1.78']


In [1]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samiraalipour/rcsb-pdb-macromolecular-structure-dataset")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 663.8 kB/s eta 0:00:00--:--

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


100%|██████████████████████████████████████| 10.6M/10.6M [00:04<00:00, 2.78MB/s]

Extracting files...


In [7]:
import pandas as pd

df = pd.read_csv('/Users/shayneskrtic/Desktop/2/rcsb_pdb_Files/rcsb_pdb_Files/rcsb_pdb_StructureData.csv')
df2 = df["Entry ID"]
print(df2)

0        6SJZ
1        5C37
2        6NNA
3        7M4C
4        7M4E
         ... 
11847    6T3M
11848    6WLX
11849    8D6J
11850    7C6B
11851    7D7U
Name: Entry ID, Length: 11852, dtype: object


In [23]:
for pdb_id in df2:
    #print(pdb_id)
    uniprot_id = pdb_to_uniprot(pdb_id)
    #protein_info = get_protein_info(uniprot_id)
    #Ec = extract_ec_numbers(protein_info)
    print(pdb_id, uniprot_id)
    #print(EC)

6SJZ ['P30419', 'Q96NN9']
5C37 ['P49327']
6NNA ['P49327']
7M4C ['Q9UGP5']
7M4E ['Q9UGP5']
7M4F ['Q9UGP5']
7M4H ['Q9UGP5']
7M4I ['Q9UGP5']
7M4L ['Q9UGP5']
5OOG ['P30043']
6QRM ['P30419', 'Q96NN9']
6SK2 ['P30419', 'Q96NN9']
6SKJ ['P30419', 'Q96NN9']
7M44 ['Q9UGP5']
7M46 ['Q9UGP5']
7M47 ['Q9UGP5']
7M4A ['Q9UGP5']
7M4B ['Q9UGP5']
7M49 ['Q9UGP5']
7M45 ['Q9UGP5']


KeyboardInterrupt: 